In [ ]:
# 라이브러리 저장
# !pip freeze > requirements.txt

In [ ]:
# 라이브러리 실행
# !pip install -r requirements.txt
# !pip install tensorflow==2.10.0

In [ ]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings(action='ignore')
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn import svm
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.model_selection import train_test_split
import random

In [ ]:
test = pd.read_excel('mental_health_medical_data_test.xlsx')
data_original = pd.read_excel('mental_health_medical_data.xlsx') #기존 데이터
data_subset = pd.read_excel('subset_data.xlsx') #확장 데이터

data = pd.concat([data_subset, data_original]) #stratify 오류 방지를 위해 각 라벨 별로 최소 2개 이상의 데이터를 가지기 위한 concat

In [ ]:
# 증상의 문자열 벗기기 함수
def listing(x):
    x = eval(x)
    return x

#모델 예측 정확도를 구하기 위한 간단한 함수
def getAccuracy(prediction, target):
  count = 0
  for x, y in zip(prediction, target):
    if x == y:
      count += 1  
  return count/len(prediction) * 100

data['symptoms'] = data['symptoms'].apply(listing)

In [ ]:
#레이블 인코딩(타겟 문자열 불가)
from sklearn.preprocessing import LabelEncoder
items = data['disease']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

print('인코딩 변환값:',labels)
print(len(labels))

인코딩 변환값: [ 0  0  0 ... 11 58  4]
79896


In [ ]:
X_train = np.array([x for x in data['symptoms']])
Y_train = np.array(labels)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True, stratify = Y_train, random_state=34) #0.8의 학습데이터, 0.2의 테스트데이터로 분리

In [ ]:
x_test

array([list(['오심', '손떨림', '가슴 두근거림', '얼굴이 화끈거림', '구강건조', '복부팽만감', '두통']),
       list(['손떨림', '가슴 답답', '얼굴이 화끈거림', '구강건조', '근육통', '권태감', '구토', '복부팽만감', '저림']),
       list(['얼굴이 화끈거림', '근육통', '구토', '가슴 쓰림', '복부팽만감', '두통', '저림']), ...,
       list(['온몸이 떨림', '오심', '손떨림', '구강건조', '근육통', '구토', '저림']),
       list(['오심', '손떨림', '가슴 답답', '얼굴이 화끈거림', '가슴 쓰림', '두통', '어지러움', '저림']),
       list(['온몸이 떨림', '오심', '손떨림', '가슴 두근거림', '얼굴이 화끈거림', '구강건조', '권태감', '두통', '저림'])],
      dtype=object)

# **모델링**

In [ ]:
#파라미터 설정
vocab_size = 178
embedding_dim = 8 # 임베딩 차원
max_length = 17      # 증상 최대 개수 
padding_type='post'

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [ ]:
word_index

{'손떨림': 1,
 '가슴 두근거림': 2,
 '얼굴이 화끈거림': 3,
 '오심': 4,
 '온몸이 떨림': 5,
 '어지러움': 6,
 '가슴 답답': 7,
 '두통': 8,
 '복부팽만감': 9,
 '구토': 10,
 '구강건조': 11,
 '권태감': 12,
 '근육통': 13,
 '저림': 14,
 '가슴 쓰림': 15,
 '생리불순': 16,
 '불면증': 17,
 '환각': 18,
 '불안': 19,
 '환청': 20,
 '숨막히는 느낌': 21,
 '가슴 통증': 22,
 '죽음에 대한 공포': 23,
 '발한': 24,
 '저혈압': 25,
 '복시': 26,
 '오한': 27,
 '청색증': 28,
 '서맥': 29,
 '기운없음': 30,
 '고혈압': 31,
 '감정 변화': 32,
 '언어장애': 33,
 '초조함': 34,
 '섬망': 35,
 '혼돈': 36,
 '환시': 37,
 '수면장애': 38,
 '감각 이상': 39,
 '기억장애': 40,
 '피로감': 41,
 '조급증': 42,
 '목의 이물감': 43,
 '근육긴장': 44,
 '실인증': 45,
 '방향감각 상실': 46,
 '실행증': 47,
 '빈뇨': 48,
 '복부 불편감': 49,
 '판단력장애': 50,
 '시력 감소': 51,
 '빈호흡': 52,
 '의식 변화': 53,
 '낮은 자존감': 54,
 '우울': 55,
 '대인관계 어려움': 56,
 '망상': 57,
 '건망증': 58,
 '인지장애': 59,
 '산만함': 60,
 '수면 무호흡': 61,
 '인내심이 부족함': 62,
 '대화에 불쑥 끼어듬': 63,
 '호흡곤란': 64,
 '와해된 언어': 65,
 '수면 주기의 변화': 66,
 '복부 통증': 67,
 '낮 시간대의 졸음': 68,
 '수면 유지가 어려움': 69,
 '흉부압박감': 70,
 '수면 중 이상한 행동': 71,
 '수면 마비': 72,
 '피해망상': 73,
 '와해된 행동': 74,
 '수면 시작이 어려움': 

In [ ]:
#테스트 데이터셋에 기존 증상을 제외한 일치하지 않는 증상 랜덤으로 추가
def add_offset(y_test, x_test):
  for disease, symptoms in zip(y_test, x_test):
    disease_name =  encoder.classes_[disease]
    ind = -1
    for index, i in enumerate(data_original['disease']):
      if disease_name == i:
        ind = index
    flag = True
    while flag:
      rand_ind = randint(1, 177)
      for k, symp in enumerate(word_index):
        if k+1 == rand_ind:
          if symp in data_original.iloc[ind]['symptoms']:
            break
          else:
            symptoms.append(symp)
            flag = False
            break

In [ ]:
add_offset(y_test, x_test)
x_test
y_test

array([37, 37, 37, ..., 37, 37, 37])

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [ ]:
test_padded

array([[ 4,  1,  2, ...,  0,  0,  0],
       [ 1,  7,  3, ...,  0,  0,  0],
       [ 3, 13, 10, ...,  0,  0,  0],
       ...,
       [ 5,  4,  1, ...,  0,  0,  0],
       [ 4,  1,  7, ...,  0,  0,  0],
       [ 5,  4,  1, ...,  0,  0,  0]], dtype=int32)

In [ ]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(64, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 8)             1424      
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                216       
                                                                 
 dense_1 (Dense)             (None, 64)                1600      
                                                                 
Total params: 3,240
Trainable params: 3,240
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
num_epochs = 20
#history = model.fit(train_padded, y_train, 
                    #epochs=1000, verbose=2)
history = model.fit(train_padded, y_train, 
                     epochs=num_epochs, verbose=2, 
                     validation_split=0.2)

Epoch 1/20
1598/1598 - 4s - loss: 0.7005 - accuracy: 0.8688 - val_loss: 0.2553 - val_accuracy: 0.9229 - 4s/epoch - 2ms/step
Epoch 2/20
1598/1598 - 3s - loss: 0.1594 - accuracy: 0.9634 - val_loss: 0.1150 - val_accuracy: 0.9719 - 3s/epoch - 2ms/step
Epoch 3/20
1598/1598 - 3s - loss: 0.0991 - accuracy: 0.9756 - val_loss: 0.0882 - val_accuracy: 0.9793 - 3s/epoch - 2ms/step
Epoch 4/20
1598/1598 - 3s - loss: 0.0782 - accuracy: 0.9812 - val_loss: 0.0692 - val_accuracy: 0.9842 - 3s/epoch - 2ms/step
Epoch 5/20
1598/1598 - 4s - loss: 0.0621 - accuracy: 0.9849 - val_loss: 0.0568 - val_accuracy: 0.9870 - 4s/epoch - 2ms/step
Epoch 6/20
1598/1598 - 3s - loss: 0.0513 - accuracy: 0.9873 - val_loss: 0.0499 - val_accuracy: 0.9872 - 3s/epoch - 2ms/step
Epoch 7/20
1598/1598 - 3s - loss: 0.0444 - accuracy: 0.9886 - val_loss: 0.0436 - val_accuracy: 0.9887 - 3s/epoch - 2ms/step
Epoch 8/20
1598/1598 - 3s - loss: 0.0400 - accuracy: 0.9895 - val_loss: 0.0406 - val_accuracy: 0.9897 - 3s/epoch - 2ms/step
Epoch 9/

In [ ]:
# 여기 부분의 train_padded 부분을 새로운 데이터 셋을 생성해주면 됨
y_prob = model.predict(test_padded, verbose=2) 
predicted = y_prob.argmax(axis=-1)
# 그리고 n번의 질문 역시도 이것을 바탕으로 진행됨

# 모델이 여러번 사용될 예정

print("학습용으로 다시 예측해봄")

encoder.classes_[predicted]

500/500 - 1s - 762ms/epoch - 2ms/step
학습용으로 다시 예측해봄


array(['신체형 장애', '신체형 장애', '신체형 장애', ..., '신체형 장애', '신체형 장애', '신체형 장애'],
      dtype=object)

In [ ]:
getAccuracy(predicted, y_test)

98.53566958698373

In [ ]:
# 질병 라벨링 순서
print(encoder.classes_)

['강박 장애' '건강염려증' '경계성 인격장애' '공황 장애' '과호흡 증후군' '광장공포증' '금단 현상' '기분부전증'
 '난독증' '뇌경색' '뚜렛 증후군' '레트 증후군' '말더듬이' '망상 장애' '뮌하우젠 증후군' '반사회성 인격장애'
 '반응성 애착장애' '발모벽' '베르니케 코시코프 증후군' '병적 도벽' '병적 방화' '분리 불안 장애' '분열정동 장애'
 '불면증' '불안 장애' '사회공포증' '산후 우울증' '섬망' '섭식 장애' '성도착증' '성인 주의력 결핍 과잉행동장애'
 '소아청소년기 우울증' '수면 무호흡증' '수면 장애' '수면보행증' '신경성 식욕부진증' '신경성 폭식증' '신체형 장애'
 '실어증' '아스퍼거 증후군' '알츠하이머병' '알코올 의존성' '알코올성 치매' '양극성 장애' '얼렌 증후군'
 '외상 후 스트레스 장애' '외상성 치매' '우울증' '의상도착증' '인터넷 중독' '자폐증 / 자폐 스펙트럼 장애' '적응 장애'
 '조현병' '주의력결핍 과잉행동장애' '지적장애' '치매' '틱 장애' '편집성 인격장애' '폐소공포증' '하지불안증후군'
 '해리 장애' '행동 및 충동 장애' '혈관성 치매' '히스테리성 인격장애']


---
# 여기서부터 추가 



더이상 입력할 증상이 없으면 종료
90%는 넘겨보자

안되면 어떤 증상을 물어보지?

현재 입력한 증상이 있는 질병중 가장 많이 출현하는 증상을 물어보자 예를 들면

"당신은 (해당 증상)을 가졌을 확률이 높습니다."

전제
1. 환자가 거짓된 증상을 입력하지 않는다
2. 오타가 나지 않는다 >> 뛰어쓰기같은건 replace(" ", "")로 처음부터 지우고 나머지는 다시 하라고 하자

#subset_data를 입력받음

In [ ]:
# #subset_data를 입력받았을 경우
# sample_symp = sample['symptoms'].apply(listing)

# for i in range(len(sample_symp)):
#   pat_word_index = sample_symp[i]
#   pat_word_array = np.array(pd.Series([pat_word_index]))

#   ##데이터를 sequence로 변환해주고 padding 해줍니다.
#   pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
#   pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

#   ##확률 계산
#   y_prob = model.predict(pat_padded, verbose=2)
#   predicted = y_prob.argmax(axis=-1)

#   print("당신의 증상은 "+str(y_prob[0][predicted[0]]*100)+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")
#   print("정답은",sample['disease'][i],"입니다.\n")

#직접 증상을 입력받음

In [ ]:
def find_mode(pat_word_index):#입력받은 증상 배열
  ##어떤 질병에 포함된 증상인지
  sample_train = []#증상의 index
  torf = True
  for i in range(len(data['symptoms'])):#전체 증상
    for j in range(len(pat_word_index)):#입력받은 증상
      torf = pat_word_index[j] in data['symptoms'][i]
      if(not torf):break
    if(torf): sample_train.append(i)
    ##모든 증상 시리즈를 다 돌면서 한 시리즈에 환자의 증상이 다 들어가 있으면 torf는 계속 true, 끝까지 true면 해당 질병을 append

  ld_symp = []#주어진 증상 배열을 포함하는 질병들의 증상 목록
  for i in data['symptoms'][sample_train]:
    for j in i:
      ld_symp.append(j)

  max = -1
  max_arr = []
  chk_symp = pat_word_index.copy()
  for i in ld_symp:#어떤 증상 i를 확인해볼까?
    count = 0
    if(i not in chk_symp):
      for j in ld_symp:#이 증상이 몇 개 있는지 배열을 돌아볼까?
        if(i == j): count += 1#i 증상이 j로 나오면 1회 추가
      
      if(count > max):#새로운 최빈값
        max_arr = [i]
        max = count

      elif(count == max):#이것도 최빈값
        max_arr.append(i)

    chk_symp.append(i)
  return max_arr

In [ ]:
# #증상 목록 출력
# for i in word_index.keys():
#   print(i)

#증상 수 초과시 종료
count = 1

#첫 증상 입력

##증상 입력받을 배열
pat_word_index = []
pat_symp = input("\n위 증상중 해당하는 증상을 하나씩 입력해주세요 (없으면 엔터) : ")
pat_word_index.append(pat_symp)

pat_word_array = np.array(pd.Series([pat_word_index]))

##데이터를 sequence로 변환해주고 padding 해줍니다.
pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

##확률 계산
y_prob = model.predict(pat_padded, verbose=0)
predicted = y_prob.argmax(axis=-1)

#두번째 이상의 증상 입력
del_token = "[']"

while (y_prob[0][predicted[0]] < 0.9) and (pat_symp !='') and (count <= vocab_size):#1. 확률이 90% 미만, 빈칸이 아님, 최대치만큼 안돌음
  target_symptoms = data_subset['symptoms'][data_subset['disease'].ne(encoder.classes_[predicted][0]).idxmin()]
  
  for t in del_token:
    target_symptoms = target_symptoms.replace(t, "")
  target_symptoms = target_symptoms.split(',')

  new = []

  for t in target_symptoms:
    if len(t) > 0 and t[0] == " ":
      t = t[1:]
    new.append(t)
  
  for t in new:
    if t in pat_word_index:
      new.remove(t)

  count = 0

  while len(new) > 0:
    count += 1
    rand_ind = random.randrange(0, len(new))

    next_symp = new[rand_ind]

    yn = input("\n" + str(count) + ". " + next_symp + " 증상이 있으십니까? (y/n): ")

    if yn == 'y':
      pat_symp = next_symp
      break;
    else:
      del new[rand_ind]

  pat_word_index.append(pat_symp)

  pat_word_array = np.array(pd.Series([pat_word_index]))

  ##데이터를 sequence로 변환해주고 padding 해줍니다.
  pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
  pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

  ##확률 계산
  y_prob = model.predict(pat_padded, verbose=0)
  predicted = y_prob.argmax(axis=-1)

  count += 1

pat_word_index = pat_word_index[:-1]
print("예측이 완료되었습니다.")



위 증상중 해당하는 증상을 하나씩 입력해주세요 (없으면 엔터) : 우울

1. 인내심이 부족함 증상이 있으십니까? (y/n): n

2. 산만함 증상이 있으십니까? (y/n): n

3. 손발을 계속 움직임 증상이 있으십니까? (y/n): y

1. 의심 증상이 있으십니까? (y/n): y

1. 대인관계 어려움 증상이 있으십니까? (y/n): y
예측이 완료되었습니다.


In [ ]:
#결과 출력

##확률이 90%가 넘어서 멈춘 경우
if(y_prob[0][predicted[0]] >0.9):
  print("당신의 증상은 "+str(y_prob[0][predicted[0]]*100)+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")

##더이상 입력을 안한 경우, 즉 입력을 완료한 경우
else:
  print("현재 입력해주신 증상으로는 "+str(y_prob[0][predicted[0]]*100)+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")

당신의 증상은 90.09684324264526%의 확률로 경계성 인격장애 입니다.


해야 할 내용

---

* 알코올성 치매, 히스테리성 인격장애는 증상이 없고, 증상이 없는 경우 확률은 30%대가 최대라 판별이 불가능한데 어떻게 해결할까 >> 일정 확률 이상의 질병들을 모두 출력할까?